# HW3 Image Classification

# Check GPU Type

In [ ]:
!nvidia-smi

# Get Data
Notes: if the links are dead, you can download the data directly from Kaggle and upload it to the workspace, or you can use the Kaggle API to directly download the data into colab.


In [ ]:
# Download Link
# Link 1 (Dropbox): https://www.dropbox.com/s/up5q1gthsz3v0dq/food-11.zip?dl=0
# Link 2 (Google Drive): https://drive.google.com/file/d/1tbGNwk1yGoCBdu4Gi_Cia7EJ9OhubYD9/view?usp=share_link
# Link 3: Kaggle Competition.

# (1) dropbox link
#!wget -O food11.zip https://www.dropbox.com/s/up5q1gthsz3v0dq/food-11.zip?dl=0

# (2) google drive link
!pip install gdown --upgrade
!gdown --id '1tbGNwk1yGoCBdu4Gi_Cia7EJ9OhubYD9' --output food11.zip

In [ ]:
! unzip food11.zip

# Import Packages

In [ ]:
_exp_name = "sample"

In [ ]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [ ]:
# 设置随机种子以保证实验可重复性
myseed = 6666  # 设置一个随机种子用于保证结果可复现

# 使CUDA运算确定性，提高实验可复现性
torch.backends.cudnn.deterministic = True
# 关闭CUDA性能优化，保持结果一致性
torch.backends.cudnn.benchmark = False

# 设置numpy的随机种子，确保涉及随机操作的结果可复现
np.random.seed(myseed)

# 设置PyTorch的随机种子，确保涉及随机操作的结果可复现
torch.manual_seed(myseed)

# 如果检测到CUDA可用，设置所有CUDA设备的种子，保证并行计算时结果的一致性
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)


# Transforms
Torchvision provides lots of useful utilities for image preprocessing, data *wrapping* as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [ ]:
# 通常情况下，我们在测试和验证阶段不需要数据增强。
# 我们这里主要做的就是调整图片大小至128x128像素，并将其转换为Tensor格式以便模型处理。
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# 然而，在测试阶段使用数据增强也是可行的。
# 你可以使用训练变换(train_tfm)生成多张不同版本的图片，然后通过集成学习方法进行测试。
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here 这里可以添加其他的数据增强方法，比如旋转、翻转等

    # ToTensor() should be the last one of the transforms.因为它将图像转换成深度学习模型需要的Tensor格式
    transforms.ToTensor(),
])


# Datasets
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [ ]:
# 定义一个自定义的FoodDataset类，继承自PyTorch的内置Dataset类
class FoodDataset(Dataset):

    # 初始化方法，用于设置类的基本属性
    def __init__(self, path, tfm=test_tfm, files=None):
        # 调用父类的初始化方法
        super(FoodDataset).__init__()
        
        # 设置数据集路径
        self.path = path
        
        # 获取当前路径下所有以.jpg结尾的文件，并按照名称排序
        self.files = sorted([os.path.join(path, x) for x in os.listdir(path) if x.endswith(".jpg")])
        
        # 如果提供了特定的文件列表，使用该列表替换默认的文件列表
        if files != None:
            self.files = files

        # 设置图像变换方法
        self.transform = tfm

    # 返回数据集中元素的数量，即图片数量
    def __len__(self):
        return len(self.files)

    # 根据索引获取数据集中的一个元素（图片及其对应的标签）
    def __getitem__(self, idx):
        # 获取指定索引的图片文件名
        fname = self.files[idx]
        
        # 打开图片文件并进行预处理
        im = Image.open(fname)
        im = self.transform(im)

        # 尝试从文件名中提取标签（假设文件名格式为数字_图片名.jpg）
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            # 如果无法解析标签（例如在测试集上，可能没有标签），则设为-1
            label = -1

        # 返回处理后的图片和对应的标签
        return im, label


# Model

In [ ]:
class Classifier(nn.Module):
    """
    这是一个图像分类器的定义，继承自PyTorch的`nn.Module`类。
    
    初始化部分定义了卷积神经网络（CNN）和全连接层（FC）的结构：
    - CNN部分用于学习图像特征，由多个卷积层、批量归一化层、ReLU激活函数和最大池化层组成。
    - FC部分将CNN提取的特征转换为预测类别，包括几个全连接层和激活函数。
    
    属性:
    - cnn: 卷积神经网络部分，负责从输入图像中提取特征。
    - fc: 全连接层部分，基于特征进行分类预测。
    
    方法:
    - __init__: 构造函数，搭建模型的架构。
    - forward: 前向传播方法，输入是图像数据，输出是预测的类别概率。
    """

    def __init__(self):
        """
        构造函数，初始化模型的各层结构。
        
        卷积层结构细节：
        1. 输入是3通道的128x128图像。
        2. 经过多次卷积（Conv2d）、批量归一化（BatchNorm2d）、ReLU激活后，逐步增加特征图的深度，
           并通过最大池化（MaxPool2d）减小空间尺寸，直至最终的特征图尺寸为512x4x4。
        
        全连接层结构：
        - 将卷积层输出平坦化后，通过两个具有ReLU激活函数的全连接层进行特征处理，
          最后一个全连接层输出为11个类别的 logits（未经过softmax处理的概率）。
        """
        super(Classifier, self).__init__()
        
        self.cnn = nn.Sequential(
            # 卷积层序列定义，每个卷积块包括卷积、批归一化、ReLU和最大池化
            nn.Conv2d(3, 64, 3, 1, 1),  # 输入通道3->输出通道64，卷积核3x3，步长1，填充1
            nn.BatchNorm2d(64),# 使用Batch Normalization层，对64个通道的特征进行规范化处理
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # 最大池化，窗口大小2x2，步长2
            
            # 后续卷积层结构类似，逐步增加输出通道数，减小特征图尺寸
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
        )
        
        self.fc = nn.Sequential(
            # 全连接层序列，用于分类
            nn.Linear(512*4*4, 1024),  # 特征图展平后的输入维度到1024维隐藏层
            nn.ReLU(),
            nn.Linear(1024, 512),     # 1024维到512维的进一步处理
            nn.ReLU(),
            nn.Linear(512, 11)        # 输出层，对应11个分类
        )

    def forward(self, x):
        """
        前向传播过程。
        
        参数:
        - x: 输入图像数据，形状为[batch_size, 3, 128, 128]。
        
        过程:
        1. 图像数据通过卷积神经网络（cnn）进行特征提取。
        2. 将卷积层输出的特征图平坦化，准备送入全连接层。
        3. 全连接层（fc）对特征进行分类处理，输出为每个类别的logits。
        
        返回:
        预测结果，形状为[batch_size, 11]，代表每个样本对于11个类别的预测分数。
        """
        out = self.cnn(x)
        # 将卷积层输出转换为一维向量，供全连接层使用
        out = out.view(out.size()[0], -1)
        # 通过全连接层得到最终的分类结果
        return self.fc(out)


# Configurations

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)

# The number of batch size.
batch_size = 64

# The number of training epochs.
n_epochs = 8

# If no improvement in 'patience' epochs, early stop.
patience = 5

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# Dataloader

In [ ]:
# Construct train and valid datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset("./train", tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset("./valid", tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

# Start Training

In [ ]:
# 初始化追踪器，这些不是参数，不应更改
stale = 0
best_acc = 0

# 遍历每个训练周期
for epoch in range(n_epochs):

    # ---------- 训练 ----------
    # 在开始训练之前，确保模型处于训练模式
    model.train()

    # 用于记录训练信息的变量
    train_loss = []
    train_accs = []

    # 遍历训练数据加载器中的每个批次
    for batch in tqdm(train_loader):

        # 每个批次包含图像数据和相应的标签
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # 将数据前向传播。确保数据和模型在同一设备上
        logits = model(imgs.to(device))

        # 计算交叉熵损失。计算损失时自动应用softmax
        loss = criterion(logits, labels.to(device))

        # 清除上一步的梯度
        optimizer.zero_grad()

        # 计算参数的梯度
        loss.backward()

        # 对梯度范数进行截断，以保证训练稳定性
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # 使用计算出的梯度更新参数
        optimizer.step()

        # 计算当前批次的准确率
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # 记录损失和准确率
        train_loss.append(loss.item())
        train_accs.append(acc)

    # 计算训练集的平均损失和准确率
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # 打印训练信息
    print(f"[ 训练 | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- 验证 ----------
    # 确保模型处于评估模式，以禁用某些模块（如dropout）并正常工作
    model.eval()

    # 用于记录验证信息的变量
    valid_loss = []
    valid_accs = []

    # 遍历验证数据加载器中的每个批次
    for batch in tqdm(valid_loader):

        # 每个批次包含图像数据和相应的标签
        imgs, labels = batch
        #imgs = imgs.half()

        # 在验证过程中不需要梯度，使用torch.no_grad()加快前向传播速度
        with torch.no_grad():
            logits = model(imgs.to(device))

        # 仍然可以计算损失（但不计算梯度）
        loss = criterion(logits, labels.to(device))

        # 计算当前批次的准确率
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # 记录损失和准确率
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # 计算验证集的平均损失和准确率
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # 打印验证信息
    print(f"[ 验证 | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # 更新日志
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ 验证 | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> 最佳")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ 验证 | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # 保存最优模型
    # 如果当前验证集的准确率优于已记录的最佳准确率，
    if valid_acc > best_acc:
        # 打印消息指示在哪个训练周期找到了更好的模型
        print(f"在第{epoch}个周期找到最佳模型，保存模型")
        # 保存模型的状态字典（即模型参数），文件名为"{_exp_name}_best.ckpt"，
        # 这样做可以确保只保留表现最好的模型，同时避免因保存过多模型导致的内存问题
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt")  
        # 更新最佳准确率为当前验证集的准确率
        best_acc = valid_acc
        # 重置“停滞计数器”stale，因为有性能提升
        stale = 0
    else:
        # 如果当前验证准确率没有提升，
        stale += 1
        # 当“停滞计数器”超过预先设定的耐心值（patience）时，
        if stale > patience:
            # 打印消息表示因连续若干周期没有性能提升，将提前终止训练
            print(f"连续{patience}个周期没有改进，提前停止")
            # 使用break语句跳出循环，结束训练
            break



# Dataloader for test

In [ ]:
# Construct test datasets.
# The argument "loader" tells how torchvision reads the data.
test_set = FoodDataset("./test", tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# Testing and generate prediction CSV

In [ ]:
# 加载之前保存的最优模型
model_best = Classifier().to(device)  # 创建一个新的Classifier实例并将其移动到设备（GPU或CPU）
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))  # 加载之前保存的模型参数
model_best.eval()  # 将模型切换到评估模式，关闭dropout等随机行为

# 初始化一个空列表，用于存储预测结果
prediction = []

# 使用torch.no_grad()上下文管理器，关闭梯度计算以提高效率
with torch.no_grad():
    # 遍历测试数据加载器中的每个批次
    for data, _ in tqdm(test_loader):  # 使用tqdm显示进度条
        # 前向传播测试数据，获取预测概率
        test_pred = model_best(data.to(device))  # 将数据移动到设备并进行预测
        # 从概率向量中获取预测类别（argmax找到最大值的索引）
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)  # 将预测概率转换回numpy数组并获取类别
        # 将预测类别添加到prediction列表中
        prediction += test_label.squeeze().tolist()  # 压缩轴并转换为列表

# prediction列表现在包含了整个测试集的预测类别


In [ ]:
# create test csv
# 定义一个函数，用于将数字转换为4位数字字符串，不足四位则在前面补零
def pad4(i):
    # 计算需要补零的位数，4 - 数字字符串的长度
    num_zeros = 4 - len(str(i))
    # 在数字前面添加num_zeros个零
    return "0" * num_zeros + str(i)

# 创建一个空的pandas DataFrame
df = pd.DataFrame()

# 使用pad4函数为DataFrame创建一个"Id"列，将0到len(test_set)范围内的整数转换为4位数字字符串
df["Id"] = [pad4(i) for i in range(len(test_set))]

# 添加"Category"列，该列包含之前预测的类别
df["Category"] = prediction

# 将DataFrame保存为CSV文件，不包含索引列
df.to_csv("submission.csv", index=False)
